In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
labels.shape

In [ ]:
train_file_location = '../input/dog-breed-identification/train/' 
train_data = labels.assign(img_path = lambda x : train_file_location + x['id'] + '.jpg')
train_data.head()

In [ ]:
train_data['breed'].nunique()

In [ ]:
train_data['img_path'][0]

In [ ]:
from keras.preprocessing.image import load_img,img_to_array, ImageDataGenerator
X = np.array([img_to_array(load_img(img,target_size = (128,128))) for img in train_data['img_path'].values.tolist()])
X.shape

In [ ]:
Y = pd.get_dummies(train_data['breed'])
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test  = train_test_split(X,Y,test_size = 0.25)

In [ ]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from keras.utils import normalize
X_train = normalize(X_train)
X_test = normalize(X_test)

In [ ]:
train_datagen = ImageDataGenerator(shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_datagen.fit(X_train)

train_generator = train_datagen.flow(X_train,y_train,batch_size = 32)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3), input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(16,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(8,(3,3)))
model.add(Activation('relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(8,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dropout(0.25))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(Y.shape[1]))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
         validation_data=(X_test, y_test),
         epochs=100,
         verbose=2)

In [ ]:
submission = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')

In [ ]:
submission.head()

In [ ]:
test_datagen = ImageDataGenerator ( 
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    '/kaggle/input/dog-breed-identification',
    target_size = (128,128),
    batch_size = 128,
    classes=['test']
)

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred.shape

In [ ]:
import re
file_list = test_set.filenames
id_list = []
for name in file_list:
    m = re.sub('test/', '', name)
    m = re.sub('.jpg', '', m)
    id_list.append(m)

In [ ]:
submission['id'] = id_list
submission.iloc[:,1:] =y_pred
submission.head()

In [ ]:
final_df = submission.set_index('id')
final_df.to_csv('Submission.csv')